<a href="https://colab.research.google.com/github/PratyushJha254/BTP/blob/main/IsingModel_Metropolis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Do the following:
1.   Code NNs using pytorch, test over NNN Ising Model.
2.   Train the model using antiferro data [Note the results].
3.   Train on ferro test on anti-ferro [Note the results].




In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import pickle
from sklearn.manifold import TSNE
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcol
import matplotlib.cm as cm
from keras.utils import to_categorical
import keras.layers as layers
from keras import Sequential
from numba import jit

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.utils as vutils
import matplotlib.pyplot as plt
import sys
from torch.autograd import Variable
from torch.autograd import grad as torch_grad
import seaborn as sns


#### Metropolis Algorithm

In [ ]:
import numpy as np
from numba import jit

@jit(nopython=True)
def data_generation_internal(l, J=1):
    # J = 1  # coupling strength
    L = l  # grid size

    temperatures = np.arange(1, 3.51, 0.1)  # 26
    ntherm = 100  # thermalization steps
    nsamples = 100  # number of samples for every starting configuration
    nbin = 10  # number of starting configurations

    n = len(temperatures) * nbin * nsamples  # total number of samples

    C = np.empty((n, L, L))
    T = np.empty(n)

    # checkerboard pattern
    x = np.zeros((L, L), dtype=np.int64)
    x[1::2, 0::2] = 1
    x[0::2, 1::2] = 1

    c = 0
    for t in temperatures:
        for i in range(nbin):
            # start with polarized state
            grid = np.ones((L, L)) * (-1)**(i % 2)
            # draw a number of sample configurations
            for j in range(nsamples):
                # perform a number of updates until thermalization
                for k in range(ntherm):
                    # Calculate neighbors without using np.roll()
                    neighbors = np.zeros((L, L))
                    neighbors[:-1, :] += grid[1:, :]
                    neighbors[1:, :] += grid[:-1, :]
                    neighbors[:, :-1] += grid[:, 1:]
                    neighbors[:, 1:] += grid[:, :-1]
                    # calculate the potential changes in energy
                    dE = 2 * J * (grid * neighbors)
                    # calculate the transition probabilities
                    p = np.exp(-dE / t)
                    # decide which transitions will occur
                    # (avoid updating neighbors using alternating checkerboard pattern)
                    grid *= 1 - 2 * np.multiply((np.random.rand(L, L) < p).astype(np.int8), x ^ (k % 2))

                C[c] = grid
                T[c] = t
                c += 1

    return C, T

In [ ]:
def data_generation(l, name, J):
    C, T = data_generation_internal(l, J)
    p = np.random.permutation(len(C))
    np.savez_compressed(name, X=C[p], y=T[p])

In [ ]:
# data_generation(10, 'data1.npz', 1)
# data_generation(20, 'data2.npz', 1)
# data_generation(30, 'data3.npz', 1)
# data_generation(40, 'data4.npz', 1)
data_generation(64, 'data_anti.npz', -1)

#### Loading the file

In [ ]:
nu_G = 10
Ny = 64
Nx = 64
nc = 1
nz = 100
ngf = 64
ndf = 64
num_epochs = 50
lr = 0.001
beta1 = 0.5
ngpu = 1
N = Ny * Nx
device = torch.device('cuda:0' if (torch.cuda.is_available() and ngpu > 0) else 'cpu')

In [ ]:
def file_loader(file_name):
    data = np.load(file_name)
    X = data['X']
    y = data['y']
    X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.15)
    Tc = 2.27 # For a square lattice following Ising Model
    y_train = Y_train > Tc
    y_test = Y_test > Tc
    y_train = to_categorical(y_train.astype(int))
    y_test = to_categorical(y_test.astype(int))
    return X_train, X_test, y_train, y_test, Y_train, Y_test

In [ ]:
def full_file_loader(file_name):
    data = np.load(file_name)
    X = data['X']
    Y = data['y']
    Tc = 2.27 # For a square lattice following Ising Model
    y = Y > Tc
    y = to_categorical(y.astype(int))
    return X, y, Y

In [ ]:
# data -

# X_train1, X_test1, y_train1, y_test1, Y_train1, Y_test1 = file_loader('data1.npz')

# X_train2, X_test2, y_train2, y_test2, Y_train2, Y_test2 = file_loader('data2.npz')

# X_train3, X_test3, y_train3, y_test3, Y_train3, Y_test3 = file_loader('data3.npz')

# X_train4, X_test4, y_train4, y_test4, Y_train4, Y_test4 = file_loader('data4.npz')

X_train, X_test, y_train, y_test, Y_train, Y_test = file_loader('data.npz')

In [ ]:
X_train_anti, X_test_anti, y_train_anti, y_test_anti, Y_train_anti, Y_test_anti = file_loader('data_anti.npz')

In [ ]:
X_anti, y_anti, Y_anti = full_file_loader('data_anti.npz')